***GENERATED CODE FOR daystolastpost PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM.***

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv("hdfs://34.83.203.43:54310"+eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ChargeDetailID0', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID0', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '3940', 'mean': '', 'stddev': '', 'min': '00026204-2f05-4d8b-891d-3e9b2e9e668a', 'max': 'fff9e00b-7619-4db2-a521-74ca70f0f086', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'ChargeDetailID0', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ChargeDetailID0')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateOfService', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfService', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'True'}, {'feature_label': 'DateOfService', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('DateOfService')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateOfEntry', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfEntry', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'True'}, {'feature_label': 'DateOfEntry', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('DateOfEntry')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DiagnosisCodeID1', 'transformation_label': 'String Indexer'}], 'feature': 'DiagnosisCodeID1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3940', 'mean': '37610.28', 'stddev': '20946.49', 'min': '13461', 'max': 'NULL', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'DiagnosisCodeID1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DiagnosisCodeID1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3940', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PatientNumber', 'threshold': 238306.9418781726, 'transformation_label': 'Binarizer'}], 'feature': 'PatientNumber', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '3940', 'mean': '238306.94', 'stddev': '158653.87', 'min': '86.0', 'max': '485396.0', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}]}, {'feature_label': 'PatientNumber', 'threshold': 238306.9418781726, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('PatientNumber')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Firstname53', 'transformation_label': 'String Indexer'}], 'feature': 'Firstname53', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3940', 'mean': '', 'stddev': '', 'min': 'Aaron', 'max': 'Yu', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'Firstname53', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Firstname53')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Lastname55', 'transformation_label': 'String Indexer'}], 'feature': 'Lastname55', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3940', 'mean': '', 'stddev': '', 'min': 'Abdelaal', 'max': 'Zuniga', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'Lastname55', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Lastname55')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderCategory', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3940', 'mean': '', 'stddev': '', 'min': 'Abdelaal, Ebtehal', 'max': 'Zuniga, Dorcas', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'ProviderCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderName', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3940', 'mean': '', 'stddev': '', 'min': 'Abdelaal, Ebtehal -46BC4C0A-E525-4793-A5F3-A91225EB39FB', 'max': 'Zuniga, Dorcas -E217C7D0-130E-47B6-AC58-59C99E52E0F6', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'ProviderName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderID65', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderID65', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3940', 'mean': '', 'stddev': '', 'min': '000c959e-d188-4091-96d1-6aa6d81562f6', 'max': 'ff2c0b65-05d1-4527-8c11-8e1bac8c12ea', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'ProviderID65', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderID65')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerName', 'transformation_label': 'String Indexer'}], 'feature': 'PayerName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3940', 'mean': '', 'stddev': '', 'min': 'AARP - Medicare Alternative Complete', 'max': 'Workers Comp', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'PayerName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'PayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3940', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'Work Comp', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'PayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerGroup', 'transformation_label': 'String Indexer'}], 'feature': 'PayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3940', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'Other Government', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'PayerGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerGroup')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
#%run daystolastpostHooks.ipynb
try:
	#sourcePreExecutionHook()

	daystolastposthdfs = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/TrainData/whitespace/whitespaceRegression.csv', 'filename': 'whitespaceRegression.csv', 'token': '', 'delimiter': ','}")
	#sourcePostExecutionHook(daystolastposthdfs)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run daystolastpostHooks.ipynb
try:
	#transformationPreExecutionHook()

	daystolastpostautofe = TransformationMain.run(daystolastposthdfs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ChargeDetailID0", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID0", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3940", "mean": "", "stddev": "", "min": "00026204-2f05-4d8b-891d-3e9b2e9e668a", "max": "fff9e00b-7619-4db2-a521-74ca70f0f086", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "days_from_dos_to_first_reason", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3940", "mean": "19.78", "stddev": "20.91", "min": "3", "max": "375", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "days_from_dos_to_first_post", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3940", "mean": "-122.99", "stddev": "2518.27", "min": "-43939", "max": "376", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "days_from_dos_to_last_post", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3940", "mean": "36.56", "stddev": "34.6", "min": "6", "max": "376", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3940", "mean": "229.11", "stddev": "431.29", "min": "7.0", "max": "11890.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Unit", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3940", "mean": "1.57", "stddev": "9.66", "min": "1.0", "max": "515.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "DateOfService", "transformation_label": "Extract Date"}], "feature": "DateOfService", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "True"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CPTCodeID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3940", "mean": "1633.66", "stddev": "1214.04", "min": "1", "max": "4880", "missing": "0"}}, {"transformationsData": [{"feature_label": "DateOfEntry", "transformation_label": "Extract Date"}], "feature": "DateOfEntry", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "True"}, {"transformationsData": [{"feature_label": "DiagnosisCodeID1", "transformation_label": "String Indexer"}], "feature": "DiagnosisCodeID1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3940", "mean": "37610.28", "stddev": "20946.49", "min": "13461", "max": "NULL", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3940", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "PatientNumber", "threshold": 238306.9418781726, "transformation_label": "Binarizer"}], "feature": "PatientNumber", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3940", "mean": "238306.94", "stddev": "158653.87", "min": "86.0", "max": "485396.0", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "Firstname53", "transformation_label": "String Indexer"}], "feature": "Firstname53", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3940", "mean": "", "stddev": "", "min": "Aaron", "max": "Yu", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "Lastname55", "transformation_label": "String Indexer"}], "feature": "Lastname55", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3940", "mean": "", "stddev": "", "min": "Abdelaal", "max": "Zuniga", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "ProviderCategory", "transformation_label": "String Indexer"}], "feature": "ProviderCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3940", "mean": "", "stddev": "", "min": "Abdelaal, Ebtehal", "max": "Zuniga, Dorcas", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "ProviderName", "transformation_label": "String Indexer"}], "feature": "ProviderName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3940", "mean": "", "stddev": "", "min": "Abdelaal, Ebtehal -46BC4C0A-E525-4793-A5F3-A91225EB39FB", "max": "Zuniga, Dorcas -E217C7D0-130E-47B6-AC58-59C99E52E0F6", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "ProviderID65", "transformation_label": "String Indexer"}], "feature": "ProviderID65", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3940", "mean": "", "stddev": "", "min": "000c959e-d188-4091-96d1-6aa6d81562f6", "max": "ff2c0b65-05d1-4527-8c11-8e1bac8c12ea", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "PayerName", "transformation_label": "String Indexer"}], "feature": "PayerName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3940", "mean": "", "stddev": "", "min": "AARP - Medicare Alternative Complete", "max": "Workers Comp", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "PayerCategory", "transformation_label": "String Indexer"}], "feature": "PayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3940", "mean": "", "stddev": "", "min": "Aetna", "max": "Work Comp", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "PayerGroup", "transformation_label": "String Indexer"}], "feature": "PayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3940", "mean": "", "stddev": "", "min": "Aetna", "max": "Other Government", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"feature": "ChargeDetailID0_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3940", "mean": "1969.5", "stddev": "1137.52", "min": "0.0", "max": "3939.0", "missing": "0"}}, {"feature": "DateOfService_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "3940", "mean": "15.23", "stddev": "8.55", "min": "1", "max": "31", "missing": "0"}}, {"feature": "DateOfService_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "3940", "mean": "2.36", "stddev": "1.93", "min": "1", "max": "12", "missing": "0"}}, {"feature": "DateOfService_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "3940", "mean": "2019.97", "stddev": "0.17", "min": "2019", "max": "2020", "missing": "0"}}, {"feature": "DateOfEntry_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "3940", "mean": "16.0", "stddev": "8.42", "min": "1", "max": "31", "missing": "0"}}, {"feature": "DateOfEntry_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "3940", "mean": "2.14", "stddev": "1.07", "min": "1", "max": "4", "missing": "0"}}, {"feature": "DateOfEntry_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "3940", "mean": "2020.0", "stddev": "0.0", "min": "2020", "max": "2020", "missing": "0"}}, {"feature": "DiagnosisCodeID1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3940", "mean": "186.38", "stddev": "256.17", "min": "0.0", "max": "1014.0", "missing": "0"}}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "3940", "mean": "0.4", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}}, {"feature": "PatientNumber_binarizer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3940", "mean": "0.49", "stddev": "0.5", "min": "0.0", "max": "1.0", "missing": "0"}}, {"feature": "Firstname53_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3940", "mean": "51.09", "stddev": "45.15", "min": "0.0", "max": "187.0", "missing": "0"}}, {"feature": "Lastname55_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3940", "mean": "69.71", "stddev": "55.4", "min": "0.0", "max": "226.0", "missing": "0"}}, {"feature": "ProviderCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3940", "mean": "73.65", "stddev": "58.37", "min": "0.0", "max": "239.0", "missing": "0"}}, {"feature": "ProviderName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3940", "mean": "73.65", "stddev": "58.37", "min": "0.0", "max": "239.0", "missing": "0"}}, {"feature": "ProviderID65_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3940", "mean": "73.65", "stddev": "58.37", "min": "0.0", "max": "239.0", "missing": "0"}}, {"feature": "PayerName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3940", "mean": "7.71", "stddev": "12.87", "min": "0.0", "max": "100.0", "missing": "0"}}, {"feature": "PayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3940", "mean": "3.49", "stddev": "3.8", "min": "0.0", "max": "19.0", "missing": "0"}}, {"feature": "PayerGroup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3940", "mean": "1.58", "stddev": "2.08", "min": "0.0", "max": "7.0", "missing": "0"}}]}))

	#transformationPostExecutionHook(daystolastpostautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run daystolastpostHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(daystolastpostautofe, ["days_from_dos_to_first_reason", "days_from_dos_to_first_post", "Amount", "Unit", "CPTCodeID", "ChargeDetailID0_stringindexer", "DateOfService_extractdate", "DateOfEntry_extractdate", "DiagnosisCodeID1_stringindexer", "Gender_stringindexer", "PatientNumber_binarizer", "Firstname53_stringindexer", "Lastname55_stringindexer", "ProviderCategory_stringindexer", "ProviderName_stringindexer", "ProviderID65_stringindexer", "PayerName_stringindexer", "PayerCategory_stringindexer", "PayerGroup_stringindexer"], "days_from_dos_to_last_post")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

